In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install evaluate accelerate bitsandbytes peft dataset datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 26.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.1/411.1 kB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.4/515.4 kB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 66.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.2/821.2 MB 2.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 4.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 103.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 67.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 46.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.0/571.0 MB 1.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.2/200.2 MB 3.

In [5]:
import os
import pickle
import zlib
from huggingface_hub import login
from datasets import load_dataset, DatasetDict, Dataset
from transformers import WhisperFeatureExtractor
from transformers import WhisperTokenizer
from transformers import WhisperProcessor
from transformers import Seq2SeqTrainingArguments
from datasets import Audio
import evaluate
from peft import prepare_model_for_kbit_training
from peft import PeftModel, PeftConfig
from transformers import WhisperForConditionalGeneration, Seq2SeqTrainer
from peft import LoraConfig, get_peft_model
from dataclasses import dataclass
from typing import Any, Dict, List, Union
import gc
import numpy as np
from tqdm import tqdm
from torch.utils.data import DataLoader
from transformers.models.whisper.english_normalizer import BasicTextNormalizer
import torch
import shutil
import subprocess
#from langdetect import detect
import subprocess
from huggingface_hub import scan_cache_dir

os.environ["HF_DATASETS_CACHE"] = "None"

login(token = "HF_TOKEN",add_to_git_credential=True)

model_name_or_path = "openai/whisper-large-v3"
#model_name_or_path="devasheeshG/whisper_large_v2_fp16_transformers"

#model_name_or_path="airesearch/wav2vec2-large-xlsr-53-th"
task = "transcribe"
dataset_name = "mozilla-foundation/common_voice_17_0"
language = "Turkish"
language_abbr = "tr"


law = DatasetDict()
med = DatasetDict()
finance = DatasetDict()
sport = DatasetDict()
other = DatasetDict()

domain_name = "/kaggle/input/health-ru/health_ru"#değişcek (law.txt)

def compress_dict(d):
    return zlib.compress(pickle.dumps(d))

# Function to decompress a dictionary
def decompress_dict(c):
    return pickle.loads(zlib.decompress(c))
law_list = list()

with open(f"{domain_name}", 'r', encoding='utf-8') as file:
    for line in file:
        law_list.append(line)
print(len(law_list))
'''
languages = ['ar', 'cs', 'de', 'el',
             'en', 'es', 'fa', 'fr', 'he', 'hi',
             'id', 'it', 'ja', 'ko',
             'nl', 'pl', 'pt',
             'ro', 'ru',
             'tr', 'uk', 'vi', 'zh-CN']
             '''
languages=["ru"]

common_voice = DatasetDict()
law_sentence = law_list[0]
print(law_sentence)
index2 = 0
dictionaries = []
for i in tqdm(languages):
    common_voice = load_dataset("mozilla-foundation/common_voice_17_0", i, split="train", use_auth_token=True, 
                                trust_remote_code=True, streaming=True)
    index = 0
    for example in common_voice:
        sentence = example["sentence"]
        if i == "en" and index >= 113800:
            break
        if i == "de" and index >= 121200:
            break
        if i == "es" and index >= 114500:
            break
        if i == "it" and index >= 131819:
            break
        if i == "fr" and index >= 128200:
            break

        if sentence == law_sentence.replace(" \n",""):
            dictionaries.append(example)
            if len(law_list) > index2 + 1:
                index2 = index2 + 1
                law_sentence = law_list[index2]

        if index % 1000 == 0:
            print("you " + str(index))
        index = index + 1


print("LEN: " + str(len(dictionaries)))


del common_voice
gc.collect()

Token is valid (permission: read).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credential helper as default.

git config --global credential.helper store

Read https://git-scm.com/book/en/v2/Git-Tools-Credential-Storage for more details.
Token has not been saved to git credential helper.
Your token has been saved to /root/.cache/huggingface/token
Login successful
680
В наших планах мы должны сделать особый упор на вопросе здоровья женщин и девочек. 



  0%|          | 0/1 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/datasets/load.py:2554: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=<use_auth_token>' instead.
  warnings.warn(

Reading metadata...: 0it [00:00, ?it/s]
Reading metadata...: 1it [00:00,  1.39it/s]
Reading metadata...: 9730it [00:00, 16113.07it/s]
Reading metadata...: 15453it [00:01, 10132.23it/s]
Reading metadata...: 26377it [00:01, 15032.01it/s]


you 0
hey
hey
hey
hey
hey
you 1000
hey
you 2000
hey
hey
you 3000
hey
hey
hey
hey
hey
you 4000
hey
hey
you 5000
hey
hey
hey
hey
hey
you 6000
hey
hey
hey
you 7000
hey
hey
hey
you 8000
hey
hey
hey
hey
hey
hey
hey
you 9000
hey
hey
hey
you 10000
hey
hey
hey
hey
you 11000
hey
you 12000
hey
hey
hey
you 13000
hey
hey
hey
hey
you 14000
hey
hey
hey
hey
hey
you 15000
hey
hey
you 16000
hey
hey
you 17000
hey
hey
you 18000
hey
hey
you 19000
hey
hey
hey
you 20000
hey
hey
hey
you 21000
hey
hey
hey
hey
hey
you 22000
hey
you 23000
hey
hey
hey
hey
hey
you 24000
hey
you 25000
hey
hey
hey
hey
you 26000
hey


100%|██████████| 1/1 [05:43<00:00, 343.49s/it]

LEN: 679


19

In [9]:
!pip install peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 4.9 MB/s eta 0:00:00a 0:00:01


In [6]:
def gen():
    for d in dictionaries[0:400]:
        yield d
        
def gen2():
    for d in dictionaries[400:]:
        yield d
        
"""def gen3():
    for d in dictionaries[800:1200]:
        yield d
        
def gen4():
    for d in dictionaries[1200:1600]:
        yield d
        
def gen5():
    for d in dictionaries[1600:2000]:
        yield d
        
def gen6():
    for d in dictionaries[2000:2400]:
        yield d
        
def gen7():
    for d in dictionaries[2400:]:
        yield d"""
"""
def gen8():
    for d in dictionaries[2800:3200]:
        yield d
def gen9():
    for d in dictionaries[3200:3600]:
        yield d
def gen10():
    for d in dictionaries[3600:]:
        yield d
"""
print(len(dictionaries))
law_data = Dataset.from_generator(gen)
law_data = law_data.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "path", "segment", "up_votes", "variant"])


law_data2 = Dataset.from_generator(gen2)
law_data2 = law_data2.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "path", "segment", "up_votes", "variant"])
"""
law_data3 = Dataset.from_generator(gen3)
law_data3 = law_data3.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "path", "segment", "up_votes", "variant"])

law_data4 = Dataset.from_generator(gen4)
law_data4 = law_data4.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "path", "segment", "up_votes", "variant"])

law_data5 = Dataset.from_generator(gen5)
law_data5 = law_data5.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "path", "segment", "up_votes", "variant"])

law_data6 = Dataset.from_generator(gen6)
law_data6 = law_data6.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "path", "segment", "up_votes", "variant"])


law_data7 = Dataset.from_generator(gen7)
law_data7 = law_data7.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "path", "segment", "up_votes", "variant"])
""""""
law_data8 = Dataset.from_generator(gen8)
law_data8 = law_data8.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "path", "segment", "up_votes", "variant"])

law_data9 = Dataset.from_generator(gen9)
law_data9 = law_data9.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "path", "segment", "up_votes", "variant"])

law_data10 = Dataset.from_generator(gen10)
law_data10 = law_data10.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "path", "segment", "up_votes", "variant"])
"""
"""train_data = law_data[10:251]
#test_data = law_data[0:100]
eval_data = law_data[:10]"""


feature_extractor = WhisperFeatureExtractor.from_pretrained(model_name_or_path)

tokenizer = WhisperTokenizer.from_pretrained(model_name_or_path, language=language, task=task)

"""processor = WhisperProcessor.from_pretrained(model_name_or_path, language=language, task=task)
"""

law_data = law_data.cast_column("audio", Audio(sampling_rate=16000))
law_data2 = law_data2.cast_column("audio", Audio(sampling_rate=16000))
"""law_data3 = law_data3.cast_column("audio", Audio(sampling_rate=16000))

law_data4 = law_data4.cast_column("audio", Audio(sampling_rate=16000))
law_data5 = law_data5.cast_column("audio", Audio(sampling_rate=16000))
law_data6 = law_data6.cast_column("audio", Audio(sampling_rate=16000))

law_data7 = law_data7.cast_column("audio", Audio(sampling_rate=16000))"""
"""law_data8 = law_data8.cast_column("audio", Audio(sampling_rate=16000))
law_data9 = law_data9.cast_column("audio", Audio(sampling_rate=16000))
law_data10 = law_data10.cast_column("audio", Audio(sampling_rate=16000))"""

def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]

    # compute log-Mel input features from input audio array
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids
    batch["labels"] = tokenizer(batch["sentence"]).input_ids
    return batch



679


Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [33]:
law_data6 = Dataset.from_generator(gen6)
law_data6 = law_data6.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "path", "segment", "up_votes", "variant"])
law_data6 = law_data6.cast_column("audio", Audio(sampling_rate=16000))


In [19]:
torch.cuda.empty_cache()


In [25]:
import torch
import gc
torch.cuda.empty_cache()
gc.collect()

20

In [7]:
law_data = law_data.map(prepare_dataset, remove_columns=law_data.column_names, num_proc=2)
law_data.save_to_disk('/kaggle/working/ru_dataset1')
del law_data
gc.collect()

Map (num_proc=2):   0%|          | 0/400 [00:00<?, ? examples/s]

Saving the dataset (0/2 shards):   0%|          | 0/400 [00:00<?, ? examples/s]

165

In [8]:
law_data2 = law_data2.map(prepare_dataset, remove_columns=law_data2.column_names, num_proc=2)
law_data2.save_to_disk('/kaggle/working/ru_dataset2')
del law_data2
gc.collect()

Map (num_proc=2):   0%|          | 0/279 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/279 [00:00<?, ? examples/s]

140

In [28]:
law_data3 = law_data3.map(prepare_dataset, remove_columns=law_data3.column_names, num_proc=2)
law_data3.save_to_disk('/kaggle/working/it_dataset3')
del law_data3
gc.collect()
law_data4 = law_data4.map(prepare_dataset, remove_columns=law_data4.column_names, num_proc=2)
law_data4.save_to_disk('/kaggle/working/it_dataset4')
del law_data4
gc.collect()
law_data5 = law_data5.map(prepare_dataset, remove_columns=law_data5.column_names, num_proc=2)
law_data5.save_to_disk('/kaggle/working/it_dataset5')
del law_data5
gc.collect()
law_data6 = law_data6.map(prepare_dataset, remove_columns=law_data6.column_names, num_proc=2)
law_data6.save_to_disk('/kaggle/working/it_dataset6')
del law_data6
gc.collect()


Map (num_proc=2):   0%|          | 0/400 [00:00<?, ? examples/s]

Saving the dataset (0/2 shards):   0%|          | 0/400 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/400 [00:00<?, ? examples/s]

Saving the dataset (0/2 shards):   0%|          | 0/400 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/400 [00:00<?, ? examples/s]

Saving the dataset (0/2 shards):   0%|          | 0/400 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/574 [00:00<?, ? examples/s]

Saving the dataset (0/2 shards):   0%|          | 0/574 [00:00<?, ? examples/s]

178

In [34]:
law_data6 = law_data6.map(prepare_dataset, remove_columns=law_data6.column_names, num_proc=2)
law_data6.save_to_disk('/kaggle/working/it_dataset6')
del law_data6
gc.collect()

Map (num_proc=2):   0%|          | 0/400 [00:00<?, ? examples/s]

Saving the dataset (0/2 shards):   0%|          | 0/400 [00:00<?, ? examples/s]

1558

In [29]:
law_data7 = law_data7.map(prepare_dataset, remove_columns=law_data7.column_names, num_proc=2)
law_data7.save_to_disk('/kaggle/working/it_dataset7')
del law_data7
gc.collect()

Map (num_proc=2):   0%|          | 0/174 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/174 [00:00<?, ? examples/s]

140

In [ ]:
law_data8 = law_data8.map(prepare_dataset, remove_columns=law_data8.column_names, num_proc=2)
law_data8.save_to_disk('/kaggle/working/en_dataset8')
del law_data8
gc.collect()
law_data9 = law_data9.map(prepare_dataset, remove_columns=law_data9.column_names, num_proc=2)
law_data9.save_to_disk('/kaggle/working/en_dataset9')
del law_data9
gc.collect()
law_data10 = law_data10.map(prepare_dataset, remove_columns=law_data10.column_names, num_proc=2)
law_data10.save_to_disk('/kaggle/working/en_dataset10')
del law_data10
gc.collect()


In [ ]:
law_data.save_to_disk('/kaggle/working/fa_dataset1')
law_data2.save_to_disk('/kaggle/working/fa_dataset2')

In [ ]:
import pickle
from datasets import load_from_disk

# Load the Dataset from the directory
data1 = load_from_disk('/kaggle/input/ar-dataset1')
data2 = load_from_disk('/kaggle/input/cs-dataset/cs_dataset')

In [9]:
import pickle
from datasets import load_from_disk

# Load the Dataset from the directory
data1 = load_from_disk('/kaggle/working/ru_dataset1')
data2 = load_from_disk('/kaggle/working/ru_dataset2')
"""data3 = load_from_disk('/kaggle/working/it_dataset3')
data4 = load_from_disk('/kaggle/working/it_dataset4')
data5 = load_from_disk('/kaggle/working/it_dataset5')
data6 = load_from_disk('/kaggle/working/it_dataset6')
data7 = load_from_disk('/kaggle/working/it_dataset7')"""



"data3 = load_from_disk('/kaggle/working/it_dataset3')\ndata4 = load_from_disk('/kaggle/working/it_dataset4')\ndata5 = load_from_disk('/kaggle/working/it_dataset5')\ndata6 = load_from_disk('/kaggle/working/it_dataset6')\ndata7 = load_from_disk('/kaggle/working/it_dataset7')"

In [10]:
import pickle
from datasets import load_from_disk

from datasets import concatenate_datasets

# Concatenate the two Datasets
combined_dataset = concatenate_datasets([data1, data2])
combined_dataset.save_to_disk('/kaggle/working/ru_dataset_med')

Saving the dataset (0/3 shards):   0%|          | 0/679 [00:00<?, ? examples/s]

In [4]:
data20 = load_from_disk('/kaggle/input/ru-dataset/ru_dataset')

In [5]:
data21 = load_from_disk('/kaggle/input/tr-dataset')
data22= load_from_disk('/kaggle/input/uk-dataset/uk_dataset')
data23= load_from_disk('/kaggle/input/vi-dataset/vi_dataset')
data24 = load_from_disk('/kaggle/input/zh-dataset/zh-dataset')

In [19]:
for i in range(1,25):
    name = f"data{i}"
    print(globals()[name])  
    
    """
    531 518 0210
    539 667 4439
    532 646 1747
    532 646 1746
    530 602 4838
    534 456 8110
    533 654 1527
    533 239 5740
    530 095 0309
    
    """


Dataset({
    features: ['input_features', 'labels'],
    num_rows: 470
})
Dataset({
    features: ['input_features', 'labels'],
    num_rows: 365
})
Dataset({
    features: ['input_features', 'labels'],
    num_rows: 2543
})
Dataset({
    features: ['input_features', 'labels'],
    num_rows: 16
})
Dataset({
    features: ['input_features', 'labels'],
    num_rows: 3869
})
Dataset({
    features: ['input_features', 'labels'],
    num_rows: 2724
})
Dataset({
    features: ['input_features', 'labels'],
    num_rows: 881
})
Dataset({
    features: ['input_features', 'labels'],
    num_rows: 2450
})
Dataset({
    features: ['input_features', 'labels'],
    num_rows: 529
})
Dataset({
    features: ['input_features', 'labels'],
    num_rows: 38
})
Dataset({
    features: ['input_features', 'labels'],
    num_rows: 43
})
Dataset({
    features: ['input_features', 'labels'],
    num_rows: 89
})
Dataset({
    features: ['input_features', 'labels'],
    num_rows: 1974
})
Dataset({
    features: 

In [ ]:
!pip install evaluate peft jiwer

In [2]:
import os
import pickle
import zlib
from huggingface_hub import login
from datasets import load_dataset, DatasetDict, Dataset
from transformers import WhisperFeatureExtractor
from transformers import WhisperTokenizer
from transformers import WhisperProcessor
from transformers import Seq2SeqTrainingArguments
from datasets import Audio
import evaluate
from peft import prepare_model_for_kbit_training
from peft import PeftModel, PeftConfig
from transformers import WhisperForConditionalGeneration, Seq2SeqTrainer
from peft import LoraConfig, get_peft_model
from dataclasses import dataclass
from typing import Any, Dict, List, Union
import gc
import numpy as np
from tqdm import tqdm
from torch.utils.data import DataLoader
from transformers.models.whisper.english_normalizer import BasicTextNormalizer
import torch
import shutil
import subprocess
#from langdetect import detect
import subprocess
from huggingface_hub import scan_cache_dir
from transformers import WhisperTokenizer

model_name_or_path = "openai/whisper-large-v3"
tokenizer = WhisperTokenizer.from_pretrained(model_name_or_path, language="ar", task="transcribe")

2024-07-31 06:30:35.202833: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-31 06:30:35.202952: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-31 06:30:35.310918: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.07k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-large-v3")
processor = WhisperProcessor.from_pretrained("openai/whisper-large-v3", task="transcribe")

In [31]:
from datasets import Dataset, concatenate_datasets

# Example combined dataset (replace this with your actual dataset)
# combined_dataset = ...  # Load your actual dataset here

# Sentence counts for each language
sentence_counts = [470, 365, 2543, 16, 3869, 2724, 881, 2979, 38, 43, 89, 1974, 103, 1, 1942, 355, 551, 334, 870, 251, 361, 8, 274]

new_datasets_train = []
new_datasets_test = []
new_datasets_eval = []

start_index = 0
for count in sentence_counts:
    end_index = start_index + count
    last_20_percent_index = int(count * 0.8)  # Get the index for the last 20%
    
    # Calculate indices for splits
    train_end_index = int(count * 0.8)  # First 80% for training
    test_end_index = int(count * 0.9)   # Next 10% for testing
    eval_end_index = count                # Remaining 10% for evaluation
   
    # Extract datasets for current language
    if count != 1:
        train_language_subset = combined_dataset.select(range(start_index, start_index+train_end_index))
        test_language_subset = combined_dataset.select(range(start_index+int(count * 0.8), start_index+int(count * 0.9)))
        eval_language_subset = combined_dataset.select(range(start_index+int(count * 0.9), start_index+count))
        print(test_language_subset)
        # Append to respective datasets
        new_datasets_train.append(train_language_subset)
        new_datasets_test.append(test_language_subset)
        new_datasets_eval.append(eval_language_subset)
    else:
        new_datasets_train.append(combined_dataset.select(range(start_index,start_index+1)))
    start_index = end_index

# Concatenate all new datasets into one
final_dataset_train = concatenate_datasets(new_datasets_train)
final_dataset_test = concatenate_datasets(new_datasets_test)
final_dataset_eval = concatenate_datasets(new_datasets_eval)

# Save or use the final datasets as needed
print("Training dataset:")
print(final_dataset_train)
print("Testing dataset:")
print(final_dataset_test)
print("Evaluation dataset:")
print(final_dataset_eval)


Dataset({
    features: ['input_features', 'labels'],
    num_rows: 47
})
Dataset({
    features: ['input_features', 'labels'],
    num_rows: 36
})
Dataset({
    features: ['input_features', 'labels'],
    num_rows: 254
})
Dataset({
    features: ['input_features', 'labels'],
    num_rows: 2
})
Dataset({
    features: ['input_features', 'labels'],
    num_rows: 387
})
Dataset({
    features: ['input_features', 'labels'],
    num_rows: 272
})
Dataset({
    features: ['input_features', 'labels'],
    num_rows: 88
})
Dataset({
    features: ['input_features', 'labels'],
    num_rows: 298
})
Dataset({
    features: ['input_features', 'labels'],
    num_rows: 4
})
Dataset({
    features: ['input_features', 'labels'],
    num_rows: 4
})
Dataset({
    features: ['input_features', 'labels'],
    num_rows: 9
})
Dataset({
    features: ['input_features', 'labels'],
    num_rows: 197
})
Dataset({
    features: ['input_features', 'labels'],
    num_rows: 10
})
Dataset({
    features: ['input_featu

In [10]:
@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)
metric = evaluate.load("wer")


In [12]:
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-large-v3", device_map="auto")
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []


model = prepare_model_for_kbit_training(model)

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.90k [00:00<?, ?B/s]

In [28]:
from tqdm import tqdm
from torch.utils.data import DataLoader
eval_dataloader = DataLoader(final_dataset_eval.select(range(2048,2049)), batch_size=5, collate_fn=data_collator)
forced_decoder_ids = processor.get_decoder_prompt_ids(language = "tr", task="transcribe")
normalizer = BasicTextNormalizer()

predictions = []
references = []
normalized_predictions = []
normalized_references = []

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model.eval()
i=0
for step, batch in enumerate(tqdm(eval_dataloader)):
    with torch.cuda.amp.autocast():
        with torch.no_grad():
            #print(law_data_backup["sentence"][i])
            #lang_sentence = backup_law_data["sentence"][i]
            i = i + 1
          
            input_features = batch["input_features"].to(device)
            #forced_decoder_ids = correct_lang_decoder(lang_sentence)
            generated_tokens = (
                    model.generate(
                    input_features=input_features,
                    forced_decoder_ids=forced_decoder_ids,
                    max_new_tokens=255,
                )
                .cpu()
                .numpy()
            )
            labels = batch["labels"].to(device)
            labels = labels.cpu().numpy()
            labels = np.where(labels != -100, labels, processor.tokenizer.pad_token_id)
            decoded_preds = processor.tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
            decoded_labels = processor.tokenizer.batch_decode(labels, skip_special_tokens=True)
            print(decoded_preds)
            print(decoded_labels)
            predictions.extend(decoded_preds)
            references.extend(decoded_labels)
            normalized_predictions.extend([normalizer(pred).strip() for pred in decoded_preds])
            normalized_references.extend([normalizer(label).strip() for label in decoded_labels])
        del generated_tokens, labels, batch, input_features
    gc.collect()
wer = 100 * metric.compute(predictions=predictions, references=references)
normalized_wer = 100 * metric.compute(predictions=normalized_predictions, references=normalized_references)
eval_metrics = {"eval/wer": wer, "eval/normalized_wer": normalized_wer}

print(f"{wer=} and {normalized_wer=}")
print(eval_metrics)

100%|██████████| 1/1 [00:04<00:00,  4.37s/it]

[' Bütçe kanun teklifi Cumhurbaşkanı tarafından hazırlanır ve meclis tarafından onaylanır.']
['Bütçe kanun teklifi Cumhurbaşkanı tarafından hazırlanır ve Meclis tarafından onaylanır.']
wer=10.0 and normalized_wer=0.0
{'eval/wer': 10.0, 'eval/normalized_wer': 0.0}


In [3]:
from datasets import load_from_disk

In [5]:
data21 = load_from_disk('/kaggle/input/fr-dataset/fr_dataset')
data22= load_from_disk('/kaggle/input/fr-dataset-son')

In [6]:
from datasets import concatenate_datasets

# Concatenate the two Datasets
combined_dataset = concatenate_datasets([data21, data22])
combined_dataset

Dataset({
    features: ['input_features', 'labels'],
    num_rows: 2979
})

In [7]:
from datasets import concatenate_datasets

# Concatenate the two Datasets
combined_dataset = concatenate_datasets([data21,data22])

print(combined_dataset)

Dataset({
    features: ['input_features', 'labels'],
    num_rows: 2979
})


In [8]:
combined_dataset.save_to_disk('/kaggle/working/fr_law_dataset')

Saving the dataset (0/10 shards):   0%|          | 0/2979 [00:00<?, ? examples/s]

In [3]:
!pip install jiwer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 13.7 MB/s eta 0:00:0000:0100:01


In [8]:
import os
import pickle
import zlib

import datasets
from huggingface_hub import login
from datasets import load_dataset, DatasetDict, Dataset
from transformers import WhisperFeatureExtractor
from transformers import WhisperTokenizer
from transformers import WhisperProcessor
from transformers import Seq2SeqTrainingArguments
from datasets import Audio
import evaluate
from peft import prepare_model_for_kbit_training
from peft import PeftModel, PeftConfig
from transformers import WhisperForConditionalGeneration, Seq2SeqTrainer
from peft import LoraConfig, get_peft_model
from dataclasses import dataclass
from typing import Any, Dict, List, Union
import gc
import numpy as np
from tqdm import tqdm
from torch.utils.data import DataLoader
from transformers.models.whisper.english_normalizer import BasicTextNormalizer
import torch
from huggingface_hub import scan_cache_dir
from sklearn.model_selection import train_test_split

os.environ["HF_DATASETS_CACHE"] = "None"

login(token = "HF_TOKEN",add_to_git_credential=True)

model_name_or_path = "openai/whisper-large-v3"
#model_name_or_path = "fine-tuned-ar"
#model_name_or_path = "fine-tuned"

#model_name_or_path="devasheeshG/whisper_large_v2_fp16_transformers"

#model_name_or_path="airesearch/wav2vec2-large-xlsr-53-th"
task = "transcribe"

law_data = datasets.load_from_disk("/kaggle/input/pt-dataset/pt_dataset")

from datasets import Dataset, concatenate_datasets

# Example combined dataset (replace this with your actual dataset)
# combined_dataset = ...  # Load your actual dataset here

# Sentence counts for each language
#sentence_counts = [470, 365, 2543, 16, 3869, 2724, 881, 2979, 38, 43, 89, 1974, 103, 1, 1942, 355, 551, 334, 870, 251,361, 8, 274]
sentence_counts = [551]

new_datasets_train = []
new_datasets_test = []
new_datasets_eval = []

start_index = 0
for count in sentence_counts:
    end_index = start_index + count
    last_20_percent_index = int(count * 0.8)  # Get the index for the last 20%

    # Calculate indices for splits
    train_end_index = int(count * 0.8)  # First 80% for training
    test_end_index = int(count * 0.9)  # Next 10% for testing
    eval_end_index = count  # Remaining 10% for evaluation

    # Extract datasets for current language
    if count != 1:
        train_language_subset = law_data.select(range(start_index, start_index + train_end_index))
        test_language_subset = law_data.select(
            range(start_index + int(count * 0.8), start_index + int(count * 0.9)))
        eval_language_subset = law_data.select(range(start_index + int(count * 0.9), start_index + count))
        print(train_language_subset)
        # Append to respective datasets
        new_datasets_train.append(train_language_subset)
        new_datasets_test.append(test_language_subset)
        new_datasets_eval.append(eval_language_subset)
    else:
        new_datasets_train.append(law_data.select(range(start_index, start_index + 1)))
    start_index = end_index

# Concatenate all new datasets into one
train_data = concatenate_datasets(new_datasets_train)
test_data = concatenate_datasets(new_datasets_test)
eval_data = concatenate_datasets(new_datasets_eval)


feature_extractor = WhisperFeatureExtractor.from_pretrained(model_name_or_path)

tokenizer = WhisperTokenizer.from_pretrained(model_name_or_path, task=task)

processor = WhisperProcessor.from_pretrained("openai/whisper-large-v3", task=task)


@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)
metric = evaluate.load("wer")


model = WhisperForConditionalGeneration.from_pretrained(model_name_or_path, load_in_8bit= True, device_map="auto")
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []


model = prepare_model_for_kbit_training(model)
def make_inputs_require_grad(module, input, output):
    output.requires_grad_(True)

model.model.encoder.conv1.register_forward_hook(make_inputs_require_grad)

config = LoraConfig(r=32, lora_alpha=64, target_modules=["q_proj", "v_proj"], lora_dropout=0.05, bias="none")

model = get_peft_model(model, config)
model.print_trainable_parameters()

from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="reach-vb/test",
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,
    learning_rate=1e-5,
    warmup_steps=50,
    eval_steps=100,  # Evaluate every 100 steps
    gradient_checkpointing=True,
    num_train_epochs=1,
    evaluation_strategy="steps",
    fp16=True,
    report_to="none",
    per_device_eval_batch_size=8,
    generation_max_length=128,
    logging_steps=1,
    lr_scheduler_type="linear",
    remove_unused_columns=False,
    label_names=["labels"],
    load_best_model_at_end=True,
    greater_is_better=False,
)


trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=train_data,
    eval_dataset=eval_data,
    data_collator=data_collator,
    tokenizer=processor.feature_extractor,
   # callbacks=[SavePeftModelCallback],
)
torch.cuda.empty_cache()
#model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

trainer.model.save_pretrained("fine-tuned-pt")
trainer.tokenizer.save_pretrained("fine-tuned-pt")
#'train_loss': 2.2751105626424155
"""



"""
"""
ar only
wer=41.84100418410041 and normalized_wer=43.36569579288026
{'eval/wer': 41.84100418410041, 'eval/normalized_wer': 43.36569579288026}

cs only
wer=10.491803278688524 and normalized_wer=6.885245901639345
{'eval/wer': 10.491803278688524, 'eval/normalized_wer': 6.885245901639345}

de only
wer=3.998334027488547 and normalized_wer=2.5758205234732032
{'eval/wer': 3.998334027488547, 'eval/normalized_wer': 2.5758205234732032}

el only
wer=20.0 and normalized_wer=6.666666666666667
{'eval/wer': 20.0, 'eval/normalized_wer': 6.666666666666667}

en only
wer=8.887130801687764 and normalized_wer=4.579755399427531
{'eval/wer': 8.887130801687764, 'eval/normalized_wer': 4.579755399427531}

es only
wer=7.092980521866961 and normalized_wer=2.4623300257258363
{'eval/wer': 7.092980521866961, 'eval/normalized_wer': 2.4623300257258363}

fa only----????????????
wer=25.04378283712785 and normalized_wer=21.015761821366024
{'eval/wer': 25.04378283712785, 'eval/normalized_wer': 21.015761821366024}

fr only

************************fine-tuned******************************************
ar: 
wer=43.93305439330544 and normalized_wer=43.36569579288026
{'eval/wer': 43.93305439330544, 'eval/normalized_wer': 43.36569579288026}

cs:
wer=10.819672131147541 and normalized_wer=7.540983606557377
{'eval/wer': 10.819672131147541, 'eval/normalized_wer': 7.540983606557377}

de:
wer=4.539775093710953 and normalized_wer=2.9912754466140425
{'eval/wer': 4.539775093710953, 'eval/normalized_wer': 2.9912754466140425}

el:
wer=20.0 and normalized_wer=6.666666666666667
{'eval/wer': 20.0, 'eval/normalized_wer': 6.666666666666667}

en:
wer=8.966244725738397 and normalized_wer=4.657819411917773
{'eval/wer': 8.966244725738397, 'eval/normalized_wer': 4.657819411917773}

es:
wer=7.239985299522235 and normalized_wer=2.499081220139655
{'eval/wer': 7.239985299522235, 'eval/normalized_wer': 2.499081220139655}

fa:
wer=21.541155866900176 and normalized_wer=19.614711033274958
{'eval/wer': 21.541155866900176, 'eval/normalized_wer': 19.614711033274958}

fr:
wer=13.718291054739653 and normalized_wer=9.309590752889722
{'eval/wer': 13.718291054739653, 'eval/normalized_wer': 9.309590752889722}

he:
wer=28.205128205128204 and normalized_wer=17.94871794871795
{'eval/wer': 28.205128205128204, 'eval/normalized_wer': 17.94871794871795}

hi:
wer=27.586206896551722 and normalized_wer=4.081632653061225
{'eval/wer': 27.586206896551722, 'eval/normalized_wer': 4.081632653061225}

id:
wer=6.25 and normalized_wer=2.5
{'eval/wer': 6.25, 'eval/normalized_wer': 2.5}

it:
wer=11.55440414507772 and normalized_wer=4.299443601416288
{'eval/wer': 11.55440414507772, 'eval/normalized_wer': 4.299443601416288}

ja:
wer=80.0 and normalized_wer=75.0
{'eval/wer': 80.0, 'eval/normalized_wer': 75.0}

ko: we cannot test korean there isn't sufficient data we used in train

nl:
wer=6.281156530408774 and normalized_wer=4.633781763826607
{'eval/wer': 6.281156530408774, 'eval/normalized_wer': 4.633781763826607}

pl:
wer=12.931034482758621 and normalized_wer=2.2988505747126435
{'eval/wer': 12.931034482758621, 'eval/normalized_wer': 2.2988505747126435}

pt:
wer=39.55555555555556 and normalized_wer=14.847161572052403
{'eval/wer': 39.55555555555556, 'eval/normalized_wer': 14.847161572052403}

ro:
wer=11.440677966101696 and normalized_wer=9.243697478991598
{'eval/wer': 11.440677966101696, 'eval/normalized_wer': 9.243697478991598}

ru:
wer=5.911949685534592 and normalized_wer=3.258145363408521
{'eval/wer': 5.911949685534592, 'eval/normalized_wer': 3.258145363408521}

tr:
wer=19.205298013245034 and normalized_wer=15.483870967741936
{'eval/wer': 19.205298013245034, 'eval/normalized_wer': 15.483870967741936}

uk:
wer=16.593886462882097 and normalized_wer=7.048458149779736
{'eval/wer': 16.593886462882097, 'eval/normalized_wer': 7.048458149779736}

vi:
wer=14.285714285714285 and normalized_wer=0.0
{'eval/wer': 14.285714285714285, 'eval/normalized_wer': 0.0}

zh-CN:
wer=85.18518518518519 and normalized_wer=71.42857142857143
{'eval/wer': 85.18518518518519, 'eval/normalized_wer': 71.42857142857143}
"""

ImportError: cannot import name 'EncoderDecoderCache' from 'transformers' (/opt/conda/lib/python3.10/site-packages/transformers/__init__.py)

In [7]:
!pip install --upgrade transformers


/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 80.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 89.4 MB/s eta 0:00:00:00:01
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.42.3
    Uninstalling transformers-4.42.3:
      Successfully uninstalled transformers-4.42.3


In [10]:
peft_model_id = "fine-tuned-pt" # Use the same model ID as before.
feature_extractor = WhisperFeatureExtractor.from_pretrained(peft_model_id)
peft_config = PeftConfig.from_pretrained(peft_model_id)
model = WhisperForConditionalGeneration.from_pretrained(
    peft_config.base_model_name_or_path,  device_map="auto"
)
model = PeftModel.from_pretrained(model, peft_model_id)
model.config.use_cache = True


eval_dataloader = DataLoader(test_data, batch_size=5, collate_fn=data_collator)
forced_decoder_ids = processor.get_decoder_prompt_ids(language="pt",task=task)
normalizer = BasicTextNormalizer()

predictions = []
references = []
normalized_predictions = []
normalized_references = []

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model.eval()
i=0
for step, batch in enumerate(tqdm(eval_dataloader)):
    with torch.cuda.amp.autocast():
        with torch.no_grad():
            #print(law_data_backup["sentence"][i])
            #lang_sentence = backup_law_data["sentence"][i]
            i = i + 1
            input_features = batch["input_features"].to(device)
            #forced_decoder_ids = correct_lang_decoder(lang_sentence)
            generated_tokens = (
                    model.generate(
                    input_features=input_features,
                    forced_decoder_ids=forced_decoder_ids,
                    max_new_tokens=255,
                )
                .cpu()
                .numpy()
            )
            labels = batch["labels"].to(device)
            labels = labels.cpu().numpy()
            labels = np.where(labels != -100, labels, processor.tokenizer.pad_token_id)
            decoded_preds = processor.tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
            decoded_labels = processor.tokenizer.batch_decode(labels, skip_special_tokens=True)
            print(decoded_labels)
            print(decoded_preds)
            predictions.extend(decoded_preds)
            references.extend(decoded_labels)
            normalized_predictions.extend([normalizer(pred).strip() for pred in decoded_preds])
            normalized_references.extend([normalizer(label).strip() for label in decoded_labels])
        del generated_tokens, labels, batch, input_features
    gc.collect()
wer = 100 * metric.compute(predictions=predictions, references=references)
normalized_wer = 100 * metric.compute(predictions=normalized_predictions, references=normalized_references)
eval_metrics = {"eval/wer": wer, "eval/normalized_wer": normalized_wer}

print(f"{wer=} and {normalized_wer=}")
print(eval_metrics)

  0%|          | 0/11 [00:00<?, ?it/s]

['exequenda', 'réus', 'lide, litígio, pleito judicial, conflituosa', 'à ordem do juízo será corrigida monetariamente', 'direito indisponível']
[' Esse é o Coen.', ' Reus.', ' Lide, litígio, pleito judicial, conflituosa.', ' A ordem do juízo será corrigida monetariamente.', ' Direito disponível.']


 18%|█▊        | 2/11 [00:20<01:35, 10.59s/it]

['cumprimento de sentença, provisório ou definitivo, na execução, resistida ou não', 'Legalização', 'intervenção federal, instrução, intimação, isonomia, togado, classista', 'judicial', 'dilação, prorrogação, diligência']
[' Cumprimento de sentença provisória ou definitiva na execução, resistida ou não.', ' Legalização.', ' Intervenção Federal, Instrução, Intimação, Isonomia, Togado, Classista.', ' Jô de Ciao.', ' Delação, prorrogação, diligência.']


 27%|██▋       | 3/11 [00:31<01:26, 10.80s/it]

...

['decisões interlocutórias', 'vinculante', 'descumprir', 'Salvo as disposições concernentes à gratuidade da justiça', 'o juiz aplicar-lhe-á a pena']
[' Decisões interlocutórias.', ' Vinculante.', ' Descumprir.', ' Salvo as disposições concernentes à gratuidade da justiça.', ' O juiz aplicaria a pena.']
['passível, ajustamento de conduta, antecipa, eficaz, descumprir, tipicidade, concretização, abstratamente', 'procuração', 'prejudicial', 'litisconsorcial', 'Os membros do Conselho podem designar expressamente um substituto em caso de ausência.']
[' Passível, ajustamento de conduta, antecipa, eficaz, descumprir, tipicidade, concretização, abstratamente.', ' Procuração.', ' prejudicial.', ' Leeds Consorcial.', ' Os membros do Conselho podem designar expressamente um substituto em caso de ausência.']


 91%|█████████ | 10/11 [01:41<00:11, 11.23s/it]

['salvo se as invocar para eximir-se de obrigação legal a todos imposta', 'Vossa inabilidade ao cargo ficou evidente', 'Não cumprirei ordem ilegal', 'nenhuma pena passará da pessoa do condenado', 'não há crime sem lei anterior que o defina, nem pena sem cominação legal']
[' salvo se as invocar para eximir-se de obrigação legal a todos imposta.', ' Vossa inabilidade ao cargo ficou evidente.', ' Não cumprirei ordem ilegal.', ' Nenhuma pena passará da pessoa do condenado', ' Não há crime sem lei anterior que o defina, nem pena sem combinação legal.']


100%|██████████| 11/11 [01:51<00:00, 10.13s/it]

wer=45.33333333333333 and normalized_wer=15.283842794759824
{'eval/wer': 45.33333333333333, 'eval/normalized_wer': 15.283842794759824}


In [ ]:
"""
he only:
wer=20.51282051282051 and normalized_wer=7.6923076923076925
{'eval/wer': 20.51282051282051, 'eval/normalized_wer': 7.6923076923076925}

hi only:
wer=20.689655172413794 and normalized_wer=2.0408163265306123
{'eval/wer': 20.689655172413794, 'eval/normalized_wer': 2.0408163265306123}

id only:
wer=8.75 and normalized_wer=5.0
{'eval/wer': 8.75, 'eval/normalized_wer': 5.0}

it only:
wer=10.414507772020725 and normalized_wer=4.704097116843703
{'eval/wer': 10.414507772020725, 'eval/normalized_wer': 4.704097116843703}

ja only:
wer=60.0 and normalized_wer=45.0
{'eval/wer': 60.0, 'eval/normalized_wer': 45.0}

nl only:
wer=5.8325024925224325 and normalized_wer=4.733432984554061
{'eval/wer': 5.8325024925224325, 'eval/normalized_wer': 4.733432984554061}

pl only:
wer=12.35632183908046 and normalized_wer=1.4367816091954022
{'eval/wer': 12.35632183908046, 'eval/normalized_wer': 1.4367816091954022}

pt only:
wer=45.33333333333333 and normalized_wer=15.283842794759824
{'eval/wer': 45.33333333333333, 'eval/normalized_wer': 15.283842794759824}


ro only:
wer=9.745762711864407 and normalized_wer=8.403361344537815
{'eval/wer': 9.745762711864407, 'eval/normalized_wer': 8.403361344537815}

ru only:
wer=6.289308176100629 and normalized_wer=3.634085213032581
{'eval/wer': 6.289308176100629, 'eval/normalized_wer': 3.634085213032581}

tr only:
wer=13.90728476821192 and normalized_wer=9.67741935483871
{'eval/wer': 13.90728476821192, 'eval/normalized_wer': 9.67741935483871}

uk only:
wer=18.340611353711793 and normalized_wer=9.691629955947137
{'eval/wer': 18.340611353711793, 'eval/normalized_wer': 9.691629955947137}

vi only:
wer=14.285714285714285 and normalized_wer=0.0
{'eval/wer': 14.285714285714285, 'eval/normalized_wer': 0.0}

zh-CN only:
wer=88.88888888888889 and normalized_wer=62.857142857142854
{'eval/wer': 88.88888888888889, 'eval/normalized_wer': 62.857142857142854}
"""

In [18]:
def calculate_domain_percentage(file_name):
    domains = [
        "Domain: agriculture",
        "Domain: education",
        "Domain: history",
        "Domain: language",
        "Domain: music"
    ]
    
    domain_counts = {domain: 0 for domain in domains}
    total_lines = 0
    
    with open(file_name, 'r') as file:
        for line in file:
            total_lines += 1
            for domain in domains:
                if domain in line:
                    domain_counts[domain] += 1
    
    if total_lines == 0:
        print("The file is empty.")
        return
    
    for domain, count in domain_counts.items():
        percentage = (count / total_lines) * 100
        print(f"{domain}: {percentage:.3f}%")

# Example usage
file_name = "/kaggle/input/dom-ar/domains_ar.txt"
calculate_domain_percentage(file_name)


Domain: agriculture: 0.004%
Domain: education: 0.032%
Domain: history: 0.106%
Domain: language: 0.004%
Domain: music: 0.004%


In [ ]:
import os
import shutil
import torch
import evaluate
import torchaudio
from datasets import load_from_disk, Dataset, DatasetDict, Audio
from transformers import (
    WhisperFeatureExtractor,
    WhisperTokenizer,
    WhisperProcessor,
    WhisperForConditionalGeneration,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    EarlyStoppingCallback
)
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model
from dataclasses import dataclass
from typing import Any, Dict, List, Union
from transformers.models.whisper.english_normalizer import BasicTextNormalizer

# --- 1. Ortam Ayarları ---
TARGET_LANG = "he"
TARGET_LANG_CODE = "hebrew"
READ_ONLY_DATASET_PATH = "/kaggle/input/he-dataset/he_dataset"
WRITABLE_DATASET_PATH = f"/kaggle/working/datasets/{TARGET_LANG}_dataset"
MODEL_OUTPUT_DIR = f"./whisper-large-v3-{TARGET_LANG}-finetuned"
CACHE_DIR = f"/kaggle/working/cache_{TARGET_LANG}"

os.makedirs(CACHE_DIR, exist_ok=True)
os.environ["HF_DATASETS_CACHE"] = CACHE_DIR
os.environ["TRANSFORMERS_CACHE"] = CACHE_DIR

model_name_or_path = "openai/whisper-large-v3"
task = "transcribe"

# --- 2. Veri Setini Kopyala ve Yükle ---
if not os.path.exists(WRITABLE_DATASET_PATH):
    shutil.copytree(READ_ONLY_DATASET_PATH, WRITABLE_DATASET_PATH)

dataset = load_from_disk(WRITABLE_DATASET_PATH)
print(dataset["train"].column_names)

# --- 3. Dataset Tipini Ayarla ve Ses Formatına Dönüştür ---
if isinstance(dataset, Dataset):
    train_test_split = dataset.train_test_split(test_size=0.2, seed=42)
    test_val_split = train_test_split["test"].train_test_split(test_size=0.5, seed=42)
    dataset = DatasetDict({
        "train": train_test_split["train"],
        "validation": test_val_split["train"],
        "test": test_val_split["test"]
    })

# `path` sütununu ses formatına çevir
dataset = dataset.cast_column("path", Audio())

# --- 4. Whisper İşleyicileri ---
feature_extractor = WhisperFeatureExtractor.from_pretrained(model_name_or_path)
tokenizer = WhisperTokenizer.from_pretrained(model_name_or_path, language=TARGET_LANG_CODE, task=task)
processor = WhisperProcessor.from_pretrained(model_name_or_path, language=TARGET_LANG_CODE, task=task)

# --- 5. Veri Hazırlama Fonksiyonu ---
def prepare_dataset(batch):
    audio = batch["path"]

    if audio is None:
        raise ValueError(f"Ses verisi yüklenemedi: {batch}")

    batch["input_features"] = feature_extractor(
        audio["array"], sampling_rate=audio["sampling_rate"]
    ).input_features[0]

    batch["labels"] = tokenizer(batch["sentence"]).input_ids
    return batch


# --- 6. Veriyi Map Et (Tokenizasyon ve Feature Extraction) ---
tokenized_dataset = dataset.map(
    prepare_dataset,
    remove_columns=list(dataset["train"].column_names),
    num_proc=1,
    batched=False,
    load_from_cache_file=False
)


# --- 7. Data Collator ---
@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any
    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]
        batch["labels"] = labels
        return batch

data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

# --- 8. Model Yükleme ve LoRA Ayarı ---
model = WhisperForConditionalGeneration.from_pretrained(model_name_or_path, load_in_8bit=True, device_map="auto")
model.config.forced_decoder_ids = processor.get_decoder_prompt_ids(language=TARGET_LANG_CODE, task=task)
model.config.suppress_tokens = []

model = prepare_model_for_kbit_training(model)
config = LoraConfig(r=16, lora_alpha=32, target_modules=["q_proj", "v_proj"], lora_dropout=0.05, bias="none")
model = get_peft_model(model, config)
model.print_trainable_parameters()

# --- 9. Eğitim Parametreleri ---
training_args = Seq2SeqTrainingArguments(
    output_dir=MODEL_OUTPUT_DIR,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=2,
    learning_rate=5e-6,
    warmup_steps=50,
    num_train_epochs=10,
    evaluation_strategy="steps",
    eval_steps=50,
    save_steps=50,
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    fp16=True,
    gradient_checkpointing=True,
    report_to="tensorboard",
    remove_unused_columns=False,
    label_names=["labels"],
)

# --- 10. Değerlendirme Fonksiyonu ---
metric = evaluate.load("wer")
normalizer = BasicTextNormalizer()

def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids
    label_ids[label_ids == -100] = tokenizer.pad_token_id
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)
    pred_str = [normalizer(text) for text in pred_str]
    label_str = [normalizer(text) for text in label_str]
    wer = 100 * metric.compute(predictions=pred_str, references=label_str)
    return {"wer": wer}

# --- 11. Trainer ve Eğitim ---
trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

print("🚀 Eğitim başlıyor...")
trainer.train()
print("✅ Eğitim tamamlandı.")

# --- 12. Modeli Kaydet ---
trainer.save_model(MODEL_OUTPUT_DIR)
print(f"💾 En iyi model {MODEL_OUTPUT_DIR} dizinine kaydedildi.")

# --- 13. Nihai Test Değerlendirmesi ---
print("\n📊 Test seti üzerinde değerlendirme:")
test_results = trainer.evaluate(eval_dataset=tokenized_dataset["test"])
print(test_results)


Map:   0%|          | 0/30 [00:00<?, ? examples/s]